# 230103 더페이스샵 크롤링

## 1.라이브러리 설치

ModuleNotFoundError: No module named 

!pip install 

In [235]:
## 셀레니움 라이브러리 설치
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [236]:
from time import sleep
import random

In [237]:
import pandas as pd 
from openpyxl.workbook import workbook

## 2.드라이버 설정

In [238]:
driver = webdriver.Chrome(executable_path = 'chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options = options)
print("##########driver open##########")

## 3.더페이스샵

### 3.1.더페이스샵 소속 브랜드 URL 가져오기

In [239]:
limit_page = 0
brand_url_list = []
while limit_page < 1:
    try:
        thefaceshop_url = "https://www.naturecollection.com/mall/index.jsp"
        driver.get(url = thefaceshop_url)
        sleep(random.randint(3, 8))

        try:
            ### 브랜드 XPATH
            brand_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div/section/div[2]/ul/li/a")
            sleep(random.randint(1, 5))
            print("브랜드 개수", len(brand_XPATH)-2)

            if len(brand_XPATH) > 1:
                limit_page = limit_page + 1

            for idx in range(1,len(brand_XPATH)-1):
                brand_url = brand_XPATH[idx].get_attribute('onclick')
                brand_url = brand_url[10:38]
                brand_url_list.append(brand_url)
        except:
            limit_page = limit_page + 1
    except:
        limit_page = limit_page + 1

print("결과 확인" , brand_url_list)

In [240]:
for idx in range(len(brand_url_list)):
    brand_url = "https://www.naturecollection.com" + brand_url_list[idx]
    print(brand_url)

## 4.드라이버 종료

In [241]:
sleep(random.randint(2, 5))
driver.close()
print("##########driver close##########")

## 5.파일 저장

In [242]:
brand_url_df = pd.DataFrame([brand_url_list])
brand_url_df = brand_url_df.transpose()
brand_url_df.columns = ['url'] 
print(brand_url_df)

In [243]:
brand_url_df.to_excel("230103_더페이스샵소속브랜드URL.xlsx")

## 6. 엑셀파일에서 브랜드 URL 가져오기

In [244]:
brand_df = pd.read_excel("C:/Users/김유진/OneDrive/바탕 화면/2022 동계인턴/크롤링/230103_더페이스샵소속브랜드URL.xlsx")
print(brand_df)

## 7. 드라이버 오픈

In [245]:
driver = webdriver.Chrome(executable_path = 'chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options = options)
print("##########driver open##########")

### 3.2. 브랜드 내 카테고리별 제품 번호 가져오기

In [246]:
brand_url_list = []
product_num_list = []
blen = len(brand_df)

for idx in range(blen):
    print(idx+1, "==================")
    brand_url = "https://www.naturecollection.com" + brand_df['url'][idx]

    try:
        driver.get(url = brand_url)
        brand_url_list.append(brand_url)
        sleep(random.randint(3,7))

        try:
            ## 클릭(카테고리)
            category_button_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div/section/div[5]/ul/li/a")
            print("카테고리:", len(category_button_XPATH))

            for bidx in range(len(category_button_XPATH)):
                category_name = category_button_XPATH[bidx].text
                sleep(random.randint(1,3))

                if "전체" in category_name:
                    continue
                elif "메이크업" in category_name:
                    continue
                elif "헤어" in category_name:
                    continue
                elif "네일" in category_name:
                    continue
                elif "향수" in category_name:
                    continue
                elif "뷰티툴즈" in category_name:
                    continue
                elif "건강식품" in category_name:
                    continue
                else:
                    category_button_XPATH[bidx].click()
                    print(category_button_XPATH[bidx].text)
                    sleep(random.randint(2,5))

                    pCount_XPATH = driver.find_element(By.XPATH, "/html/body/div[2]/div/section/div[6]/div/div[1]/span")
                    pCount = int(pCount_XPATH.text)
                    print(pCount)

                    limit = 0
                    while limit < 1:
                        ## 클릭(더보기)
                        more_button_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div/section/div[7]/div[2]/div[2]/a")
                        try:
                            more_button_XPATH[0].click()
                            sleep(random.randint(3,7))
                        
                        except:
                            print("more_buttom click error")
                            limit = limit + 1
                            break

                    try:
                        ## 제품 번호 추출
                        product_num_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div/section/div[7]/div[2]/div[1]/ul/li/a")
                        print(len(product_num_XPATH))

                        for count in range(pCount):
                            product_num = product_num_XPATH[count].get_attribute('onclick')
                            product_num = product_num[13:21]
                            product_num_list.append(product_num)
                            print(product_num)

                    except:
                        print("product_num get error")
        except:
            print("category click error")
    except:
        print("brand_url get error")

## 9.드라이버 종료

In [247]:
sleep(random.randint(2, 5))
driver.close()
print("##########driver close##########")

## 10. 파일 저장 (제품 번호)

In [248]:
product_num_df = pd.DataFrame([product_num_list])
product_num_df = product_num_df.transpose()
product_num_df.columns = ['url'] 
#print(product_num_df)

In [249]:
product_num_df.to_excel("230104_더페이스샵 제품번호.xlsx")

## 11. 엑셀 파일에서 제품 번호 가져오기

In [250]:
product_num_df = pd.read_excel("C:/Users/김유진/OneDrive/바탕 화면/2022 동계인턴/크롤링/230104_더페이스샵 제품번호.xlsx")
print(product_num_df)

     Unnamed: 0       url
0             0  AF015652
1             1  AF015634
2             2  AF015611
3             3  AF015601
4             4  AF015589
..          ...       ...
824         824  AF014820
825         825  AF014230
826         826  AF011241
827         827  AF010992
828         828  AF013283

[829 rows x 2 columns]


## 12. 드라이버 오픈

In [251]:
driver = webdriver.Chrome(executable_path = 'chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options = options)
print("##########driver open##########")

## 3.3. 상세 페이지 접속

In [252]:
product_url_list = []
category_list = []
brand_name_list = []
product_name_list = []
product_img_list = []
origin_price_list = []
sale_price_list = []
# sale_time_list = []
product_detail_img_list = []
## 구매정보 클릭 후, 수집되는 리스트
ml_info_list = []
REC_skin_type_list = []
use_possible_list = []
how_to_use_list = []
oem_list = []
ingred_list = []
functional_test_list = []

plen = len(product_num_df);
for idx in range(plen):
    print(idx+1, "==================")
    product_url = "https://www.naturecollection.com/mall/product/product-view.jsp?dpid=" + product_num_df['url'][idx]

    try:
        driver.get(url = product_url)
        ## 해당 URL 주소
        product_url_list.append(product_url)
        sleep(random.randint(3, 7))

        try:
            ## 카테고리 
            category_XPATH = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[1]/ul/li[3]/span")
            category = category_XPATH.text
            print("카테고리:", category)
            category_list.append(category)
            sleep(random.randint(1,4))
        except:
            category_list.append("FALSE")
            sleep(random.randint(1,4))  

        try:
            ## 브랜드
            brand_name_XPATH = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[2]/div/form/div/div[1]/a")
            brand_name = brand_name_XPATH.text
            print("브랜드명:", brand_name)
            brand_name_list.append(brand_name)
            sleep(random.randint(1,4))
        except:
            brand_name_list.append("FALSE")
            sleep(random.randint(1,4))  

        try:
            ## 제품
            product_name_XPATH = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[2]/div/form/div/div[1]/p")
            product_name = product_name_XPATH.text
            print("제품명:", product_name)
            product_name_list.append(product_name)
            sleep(random.randint(1,4))      
        except:
            product_name_list.append("FALSE")
            sleep(random.randint(1,4))  

        try:
            ## 가격(원가, 할인가)
            price_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[2]/div/form/div/div[1]/div[2]/span")
            if plen == 2:
                origin_price= driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[2]/div/form/div/div[1]/div[2]/span[2]/span").text
                print(origin_price)
                sale_price = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[2]/div/form/div/div[1]/div[2]/strong/span").text
                ### 단위 제거
                sale_price = sale_price.replace("\"", "")
                sale_price = sale_price.replace("원", "")
                print("원가:", origin_price, "-->", " 할인가:", sale_price)
                origin_price_list.append(origin_price)
                sale_price_list.append(sale_price)
            elif plen == 0:
                origin_price = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[2]/div/form/div/div[1]/div[2]/strong/span").text
                origin_price = origin_price.replace("원", "")
                print("원가:", origin_price)
                origin_price_list.append(origin_price)
            else:
                continue
        except:
            origin_price_list.append("FALSE")
            sale_price_list.append("FALSE")
            sleep(random.randint(1,4))
            
        try:
            ## 제품 이미지
            product_img_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[2]/div/div/div[2]/div[1]/ul/li/img")
            if len(product_img_XPATH) > 1:
                product_img = ""
                ### 이미지가 여러개인 경우
                for img in range(len(product_img_XPATH)):
                    product_img = product_img + product_img_XPATH[img].get_attribute('src')
                    product_img = product_img + "||^^||"
                ## product_img = product_img.replace("/images/goods/85/", "/images/goods/550/")     // 이미지 크기 확대
                print("제품 이미지 경로:", product_img)
                product_img_list.append(product_img)
                sleep(random.randint(1,4))
            elif len(product_img_XPATH) == 1:
                product_img = product_img_XPATH[0].get_attribute('src')
                # product_img = product_img.replace("/images/goods/85/", "/images/goods/550/")
                print("제품 이미지 경로:", product_img)
                product_img_list.append(product_img)
                sleep(random.randint(1,4))
        except:
            product_img_list.append("FALSE")
            sleep(random.randint(1,4))  

        try:
            ## 상세설명(이미지)
            product_detail_img_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[8]/p/img")#(By.CLASS_NAME, "contEditor")[0].find_elements(By.TAG_NAME, "img")
            sleep(random.randint(1,4)) 
            product_detail_img = ""
            if len(product_detail_img_XPATH) > 1:
                for img in range(len(product_detail_img_XPATH)):
                    product_detail_img = product_detail_img + product_detail_img_XPATH[img].get_attribute('src')
                    product_detail_img = product_detail_img + "||^^||"
                print("상세설명 이미지:", product_detail_img)
                product_detail_img_list.append(product_detail_img)
            elif len(product_detail_img_XPATH) == 1:
                product_detail_img = product_detail_img_XPATH[0].get_attribute('src')
                print("상세설명 이미지:", product_detail_img)
                product_detail_img_list.append(product_detail_img)
        except:
            product_detail_img_list.append("FALSE")
            sleep(random.randint(1,4))

        try:
            ## 클릭(유의사항)
            info_button_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[5]/ul/li[2]/a/span[2]")                          
            info_button_XPATH[0].click()
            sleep(random.randint(2,5))

            try:
                ## 용량
                ml_info_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[7]/table/tbody/tr[1]/td")
                ml_info = ml_info_XPATH[0].text
                print("용량 또는 증량:", ml_info)
                ml_info_list.append(ml_info)
                sleep(random.randint(2,5))
            except:
                ml_info_list.append("FALSE")
                sleep(random.randint(2,5))

            try:
                ## 제품주요사항
                REC_skin_type_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[7]/table/tbody/tr[2]/td")
                REC_skin_type = REC_skin_type_XPATH[0].text
                print("제품 주요 사항:", REC_skin_type)
                REC_skin_type_list.append(REC_skin_type)
                sleep(random.randint(2,5))
            except:
                REC_skin_type_list.append("FALSE")
                sleep(random.randint(2,5))

            try:
                ## 사용기간(개봉 후 사용기간)
                use_possible_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[7]/table/tbody/tr[3]/td")
                use_possible = use_possible_XPATH[0].text
                print("사용기간:", use_possible)
                use_possible_list.append(use_possible)
                sleep(random.randint(2,5))
            except:
                use_possible_list.append("FALSE")
                sleep(random.randint(2,5))

            try:
                ## 사용방법
                how_to_use_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[7]/table/tbody/tr[4]/td")
                how_to_use = how_to_use_XPATH[0].text
                print("사용방법:", how_to_use)
                how_to_use_list.append(how_to_use)
                sleep(random.randint(2,5))
            except:
                how_to_use_list.append("FALSE")
                sleep(random.randint(2,5))

            try:
                ## 화장품제조업자/화장품책임판매업자
                oem_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[7]/table/tbody/tr[5]/td")
                oem = oem_XPATH[0].text
                print("화장품제조업자:", oem)
                oem_list.append(oem)
                sleep(random.randint(2,5))
            except:
                oem_list.append("FALSE")
                sleep(random.randint(2,5))
            
            try:
                ingred_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[7]/table/tbody/tr[6]/td")
                ingred = ingred_XPATH[0].text
                print("전성분:", ingred)
                ingred_list.append(ingred)
                sleep(random.randint(2,5))
            except:
                ingred_list.append("FALSE")
                sleep(random.randint(2,5))

            try:
                functional_test_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[7]/table/tbody/tr[7]/td")
                functional_test = functional_test_XPATH[0].text
                print("기능성 화장품 식품의약처 심사 여부:", functional_test)
                functional_test_list.append(functional_test)
                sleep(random.randint(2,5))
            except:
                functional_test_list.append("FALSE")
                sleep(random.randint(2,5))

        except:
            ml_info_list.append("FALSE")
            REC_skin_type_list.append("FALSE")
            use_possible_list.append("FALSE")
            how_to_use_list.append("FALSE")
            oem_list.append("FALSE")
            ingred_list.append("FALSE")
            functional_test_list.append("FALSE")
            sleep(random.randint(2,5))

    except:
        product_url_list.append("FALSE")
        category_list.append("FALSE")
        brand_name_list.append("FALSE")
        product_name_list.append("FALSE")
        product_img_list.append("FALSE")
        origin_price_list.append("FALSE")
        sale_price_list.append("FALSE")
        # sale_time_list.append("FALSE")
        product_detail_img_list.append("FALSE")
        sleep(random.randint(1,4))  

## 14.드라이버 종료


In [253]:
sleep(random.randint(3, 5))
driver.close()

In [254]:
product_df = pd.DataFrame([product_url_list, category_list, 
                           brand_name_list, product_name_list, product_img_list,
                           origin_price_list, sale_price_list, 
                           product_detail_img_list,
                           ## 클릭 (구매 정보)
                           ml_info_list, REC_skin_type_list, use_possible_list, how_to_use_list,
                           oem_list, ingred_list, functional_test_list])  #sale_time_list,
product_df = product_df.transpose()
product_df.columns = ['url', 'category', 
                      'brand_name', 'product_name', 'product_img_path',
                      'origin_price', 'sale_price', 
                      'product_detail_img',
                      'ml_info', 'rec_skin_type', 'use_possible', 'how to use',
                      'oem_info', 'ingred', 'functional_test']    #'sale_time',
print(product_df)

### 6.3.파일 저장

In [255]:
product_df.to_excel("230109_더페이스샵제품정보.xlsx")

## 15. 드라이버 오픈

In [256]:
driver = webdriver.Chrome(executable_path = 'chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options = options)
print("##########driver open##########")

##########driver open##########


## 3.4. 리뷰 페이지 접속

In [257]:
product_url_list = []
user_ID_list = []
user_spec_list = []
rating_list = []
review_date_list = []
review_cotents_list = []
review_img_list = []

plen = len(product_num_df);
for idx in range(4):#plen):
    print(idx+1, "==================")
    product_url = "https://www.naturecollection.com/mall/product/product-view.jsp?dpid=" + product_num_df['url'][idx]

    try:
        driver.get(url = product_url)
        ## 해당 URL 주소
        product_url_list.append(product_url)
        sleep(random.randint(3, 7))

        try:
            ## 클릭(리뷰)
            review_button_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[5]/ul/li[3]/a")                          
            review_button_XPATH[0].click()
            sleep(random.randint(2,5))

            try:
                ## 사용자 아이디
                user_ID_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li/div[1]/div[3]/p[1]/span[1]")
                review_num = len(user_ID_XPATH)
                print("제품별 리뷰 개수:", review_num)

                if review_num >= 100:
                    for n in range(100):
                        user_ID = user_ID_XPATH[n].text
                        print("사용자 아이디:", user_ID)
                        user_ID_list.append(user_ID)

                        if n != 0:
                            product_url_list.insert(idx+n, product_url)
                        sleep(random.randint(2,5))

                        try:
                            ## 사용자 특징(나이/건성)
                            tmp = "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li[" + str(n+1) + "]/div[1]/div[3]/p[2]/span"
                            user_spec_XPATH = driver.find_elements(By.XPATH, tmp)
                            slen = len(user_spec_XPATH)
                            print("사용자 특징 개수:", slen)

                            user_spec = ""
                            if slen > 1:
                                for spec in range(slen):
                                    user_spec = user_spec + user_spec_XPATH[spec].text
                                    user_spec = user_spec + "||^^||"
                                print("사용자 특징:", user_spec)
                                user_spec_list.append(user_spec)
                                sleep(random.randint(2,5))
                            elif slen == 1:
                                user_spec = user_spec_XPATH[0].text
                                print("사용자 특징:", user_spec)
                                user_spec_list.append(user_spec)
                                sleep(random.randint(2,5))
                        except:
                            user_spec_list.append("FALSE")
                            sleep(random.randint(2,5))

                        try:
                            ## 별점
                            rating_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li["+str(n+1)+"]/div[1]/div[1]/span/span")
                            rating = rating_XPATH[0].get_attribute('style')
                            rating = rating.replace("width: 100%;", "5")
                            rating = rating.replace("width: 80%;", "4")
                            rating = rating.replace("width: 60%;", "3")
                            rating = rating.replace("width: 40%;", "2")
                            rating = rating.replace("width: 20%;", "1")
                            print("별점:", rating)
                            rating_list.append(rating)
                            sleep(random.randint(2,5))
                        except:
                            rating_list.append("FALSE")
                            sleep(random.randint(2,5))

                        try:
                            ## 리뷰 작성 날자
                            review_date_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li["+str(n+1)+"]/div[1]/div[3]/p[1]/span[2]")
                            review_date = review_date_XPATH[0].text
                            print("리뷰 작성 날자:", review_date)
                            review_date_list.append(review_date)
                            sleep(random.randint(2,5))
                        except:
                            review_date_list.append("FALSE")
                            sleep(random.randint(2,5))

                        try:
                            ## 사용자 리뷰 내용
                            review_cotents_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li["+str(n+1)+"]/div[2]/a/p")
                            review_cotents = review_cotents_XPATH[0].text
                            print("사용자 리뷰 내용:", review_cotents)
                            review_cotents_list.append(review_cotents)
                            sleep(random.randint(2,5))
                        except:
                            review_cotents_list.append("FALSE")
                            sleep(random.randint(2,5))
                    break

                elif review_num >= 1:
                    for n in range(review_num):
                        user_ID = user_ID_XPATH[n].text
                        print("사용자 아이디:", user_ID)
                        user_ID_list.append(user_ID)

                        if n != 0:
                            product_url_list.insert(idx+n, product_url)
                        sleep(random.randint(2,5))

                        try:
                            ## 사용자 특징(나이/건성)
                            tmp = "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li[" + str(n+1) + "]/div[1]/div[3]/p[2]/span"
                            user_spec_XPATH = driver.find_elements(By.XPATH, tmp)
                            slen = len(user_spec_XPATH)
                            print("사용자 특징 개수:", slen)

                            user_spec = ""
                            if slen > 1:
                                for spec in range(slen):
                                    user_spec = user_spec + user_spec_XPATH[spec].text
                                    user_spec = user_spec + "||^^||"
                                print("사용자 특징:", user_spec)
                                user_spec_list.append(user_spec)
                                sleep(random.randint(2,5))
                            elif slen == 1:
                                user_spec = user_spec_XPATH[0].text
                                print("사용자 특징:", user_spec)
                                user_spec_list.append(user_spec)
                                sleep(random.randint(2,5))
                        except:
                            user_spec_list.append("FALSE")
                            sleep(random.randint(2,5))

                        try:
                            ## 별점
                            rating_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li["+str(n+1)+"]/div[1]/div[1]/span/span")
                            rating = rating_XPATH[0].get_attribute('style')
                            rating = rating.replace("width: 100%;", "5")
                            rating = rating.replace("width: 80%;", "4")
                            rating = rating.replace("width: 60%;", "3")
                            rating = rating.replace("width: 40%;", "2")
                            rating = rating.replace("width: 20%;", "1")

                            print("별점:", rating)
                            rating_list.append(rating)
                            sleep(random.randint(2,5))
                        except:
                            rating_list.append("FALSE")
                            sleep(random.randint(2,5))

                        try:
                            ## 리뷰 작성 날자
                            review_date_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li["+str(n+1)+"]/div[1]/div[3]/p[1]/span[2]")
                            review_date = review_date_XPATH[0].text
                            print("리뷰 작성 날자:", review_date)
                            review_date_list.append(review_date)
                            sleep(random.randint(2,5))
                        except:
                            review_date_list.append("FALSE")
                            sleep(random.randint(2,5))

                        try:
                            ## 사용자 리뷰 내용
                            review_cotents_XPATH = driver.find_elements(By.XPATH, "/html/body/div[2]/div[2]/section/div/div[10]/div[2]/ul/li["+str(n+1)+"]/div[2]/a/p")
                            review_cotents = review_cotents_XPATH[0].text
                            print("사용자 리뷰 내용:", review_cotents)
                            review_cotents_list.append(review_cotents)
                            sleep(random.randint(2,5))
                        except:
                            review_cotents_list.append("FALSE")
                            sleep(random.randint(2,5))
                else:
                    user_ID_list.append("NONE")
                    user_spec_list.append("NONE")
                    rating_list.append("NONE")
                    review_date_list.append("NONE")
                    review_cotents_list.append("NONE")
                    print("no review")
            except:
                user_ID_list.append("FALSE")
                sleep(random.randint(2,5))

        except:
            user_ID_list.append("FALSE")
            user_spec_list.append("FALSE")
            rating_list.append("FALSE")
            review_date_list.append("FALSE")
            review_cotents_list.append("FALSE")
            sleep(random.randint(2,5))

    except:
        product_url_list.append("FALSE")
        sleep(random.randint(1,4))  

1 ==================
제품별 리뷰 개수: 0
no review
2 ==================
제품별 리뷰 개수: 0
no review
3 ==================
제품별 리뷰 개수: 1
사용자 아이디: KA_1********
사용자 특징 개수: 1
사용자 특징: 30대
별점: 5
리뷰 작성 날자: 23.01.04
사용자 리뷰 내용: 너무 좋아요
잘 쓸게요~
감사합니다
나한테 잘 맞아요~
지인에게도 강추하고싶어요
이 제품만 쓰게되고 찾게되고
향 만족도 아주 만족해요
자극도 자극이 없어요
적당해요
4 ==================
제품별 리뷰 개수: 3
사용자 아이디: KA_1********
사용자 특징 개수: 1
사용자 특징: 30대
별점: 5
리뷰 작성 날자: 23.01.07
사용자 리뷰 내용: 겨울철 닥터벨머 시카 리커버리크림을 애용하는데 저렴하게 잘샀네요. 덤으로 어드밴스드 시카 펩타이드 앰플까지 너무 좋은 가격에 구매하게 되었어요. 시카 리커버리크림은 매번 자타입 용기만 사용했는데 튜브타입이라 사용하기 편하고 위생적이네요
사용자 아이디: KA_1********
사용자 특징 개수: 1
사용자 특징: 30대
별점: 5
리뷰 작성 날자: 23.01.03
사용자 리뷰 내용: 닥터벨머는 순하지만 기능도 좋아서 제가 지인들한테 추천하는 제품인데요!^^

토너 바르고 크림 바르기 전에 에센스는 꼭 발라주고 있는데 시카 펩타이트 앰플은 바르면 쫀쫀하게 싹 흡수가 돼서 너무 좋더라구요!

여행갈 때 샘플 챙겨갔는데 지인(민감성 피부)도 써보더니 이거 어디꺼냐고 물어볼 정도였으면 말 다했죠ㅎㅎ

샘플 받은 거 쓰다가 마침 기획으로 나온 게 있어 구입했는데 팩까지 들어있네요~ 피부 컨디션 진짜 안 좋을 때 쓰면 효과 백방일 듯 합니다!^^
잘쓸게요♡
사용자 아이디: 11ya*****
사용자 특징 개수: 6
사용자 특징: 30대||^^||건성||^^||민감성||^^||미백||^^||주름개선/탄력||^^||민감성/트러블||^^||
별점: 4


## 17.드라이버 종료

In [258]:
sleep(random.randint(3, 5))
driver.close()

In [259]:
review_df = pd.DataFrame([product_url_list, user_ID_list, user_spec_list, 
                        rating_list, review_date_list, review_cotents_list])
review_df = review_df.transpose()
review_df.columns = ['url', 'user_ID', 'user_spec',
                    'rating', 'review_date', 'review_cotents']
print(review_df)

                                                 url       user_ID  \
0  https://www.naturecollection.com/mall/product/...          NONE   
1  https://www.naturecollection.com/mall/product/...          NONE   
2  https://www.naturecollection.com/mall/product/...  KA_1********   
3  https://www.naturecollection.com/mall/product/...  KA_1********   
4  https://www.naturecollection.com/mall/product/...  KA_1********   
5  https://www.naturecollection.com/mall/product/...     11ya*****   

                                           user_spec rating review_date  \
0                                               NONE   NONE        NONE   
1                                               NONE   NONE        NONE   
2                                                30대      5    23.01.04   
3                                                30대      5    23.01.07   
4                                                30대      5    23.01.03   
5  30대||^^||건성||^^||민감성||^^||미백||^^||주름개선/탄력||^^|...      4

## 18. 파일 저장

In [261]:
review_df.to_excel("230110_더페이스샵 리뷰.xlsx")